# How to add a human-in-the-loop for tools

There are certain tools that we don't trust a model to execute on its own. One thing we can do in such situations is require human approval before the tool is invoked.

Steps for Adding Human-in-the-Loop (HITL)

2. Define Tools

Here are two example tools:

* count_emails: Counts the number of emails received in the last n days.
* send_email: Sends an email to a recipient.


In [6]:
from langchain_core.tools import tool

@tool
def count_emails(last_n_days: int) -> int:
    """Counts the number of emails. Dummy implementation."""
    return last_n_days * 2

@tool
def send_email(message: str, recipient: str) -> str:
    """Sends an email. Dummy implementation."""
    return f"Email sent to {recipient}: {message}"


3. Set Up LLM and Bind Tools

Use a LangChain-compatible LLM like OpenAI's ChatGPT and bind the tools to it:

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [count_emails, send_email]
llm_with_tools = llm.bind_tools(tools)


4. Define Human Approval Logic

This function intercepts the tool calls proposed by the LLM and prompts a human for approval.

In [8]:
import json

class NotApproved(Exception):
    """Custom exception raised when tool execution is not approved."""

def human_approval(tool_calls):
    """Ask for human approval before executing tools."""
    for tool_call in tool_calls:
        # Display proposed tool invocation details
        print("Tool invocation request:")
        print(json.dumps(tool_call, indent=2))
        
        # Get human approval
        response = input("\nApprove this tool invocation? (yes/no): ")
        if response.lower() not in {"yes", "y"}:
            raise NotApproved(f"Tool invocation not approved: {tool_call}")
    return tool_calls


5. Create the Chain

Combine the LLM and tools with the human approval step in a chain.

In [9]:
def execute_chain(input_text: str):
    # Step 1: Generate tool invocation proposals from LLM
    ai_message = llm_with_tools.invoke(input_text)
    
    # Step 2: Get human approval
    try:
        approved_calls = human_approval(ai_message.tool_calls)
    except NotApproved as e:
        print(e)
        return
    
    # Step 3: Execute approved tools
    results = []
    for tool_call in approved_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool = next(t for t in tools if t.name == tool_name)
        results.append(tool.invoke(tool_args))
    
    return results


6. Example Usage

You can now run the chain with human approval included:

In [10]:
# Input asking about email count in the past few days
print(execute_chain("How many emails did I receive in the last 5 days?"))

# Input for sending an email
print(execute_chain("Send an email to john@example.com saying 'Meeting at 3 PM.'"))


Tool invocation request:
{
  "name": "count_emails",
  "args": {
    "last_n_days": 5
  },
  "id": "call_pu6EnY5PCXAX5pLYatwlyBiV",
  "type": "tool_call"
}
[10]
Tool invocation request:
{
  "name": "send_email",
  "args": {
    "message": "Meeting at 3 PM.",
    "recipient": "john@example.com"
  },
  "id": "call_s1Hecy1ro2EyFimOnl3vI9n3",
  "type": "tool_call"
}
['Email sent to john@example.com: Meeting at 3 PM.']


# Execution Workflow
1. User Input: The user inputs a query (e.g., "How many emails did I receive in the last 5 days?").
2. Tool Invocation Proposal:
* The LLM generates tool calls (e.g., count_emails(last_n_days=5)).
3. Human Approval:
* A prompt is displayed to approve or reject the tool call.
4. Tool Execution:
* If approved, the tool is executed; otherwise, the process stops, and an exception is raised.


# Advantages

Prevents unintended or unsafe tool execution.

Enables humans to remain in control during critical decisions.

Easily extendable for additional tools or more complex approval logic.